In [2]:
import numpy as np
from scipy import stats

In [19]:
def simulate_power_optimized(n, k, p_null=0.5, p_alt=0.52, alpha=0.05, n_sim=1000):
    """
    Optimized simulation of power for given sample size and number of pairs.
    """
    # Simulate all choices at once: n_sim simulations x n participants x k pairs
    choices = np.random.binomial(1, p_alt, size=(n_sim, n, k))
    
    # Calculate the proportion of Option A choices per participant for each simulation
    prop_A = choices.mean(axis=2)  # Shape: (n_sim, n)
    
    # Calculate the mean proportion for each simulation
    mean_prop_A = prop_A.mean(axis=1)  # Shape: (n_sim,)
    
    # Calculate the standard error for each simulation
    se_prop_A = prop_A.std(axis=1, ddof=1) / np.sqrt(n)
    
    # Calculate t-statistics for all simulations
    t_stats = (mean_prop_A - p_null) / se_prop_A
    
    # Calculate two-tailed p-values
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-1))
    
    # Calculate power as the proportion of p-values below alpha
    power = np.mean(p_values < alpha)
    return power

In [21]:
simulate_power_optimized(500, 10)

np.float64(0.814)

In [22]:
import duckdb

In [23]:
con = duckdb.connect()

In [25]:
df = con.execute("SELECT * FROM 'data/nyt_archive_all.parquet' ").fetchdf()

In [30]:
top_desks = df.news_desk.value_counts().head(20)

In [31]:
top_desks.sum() / df.shape[0]

np.float64(0.887894403760962)

In [35]:
in_desks = df[df.news_desk.isin(top_desks.index)]

In [44]:
sample = in_desks.groupby('news_desk').apply(lambda x: x.sample(n=3, random_state=42))

/var/folders/m8/v1brzj4s56g48wwmpht61td40000gq/T/ipykernel_2409/1285063922.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample = in_desks.groupby('news_desk').apply(lambda x: x.sample(n=3, random_state=42))


In [49]:
headline_sample = sample.headline.tolist()